<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/PyTorch_Tips_Details/r_TrainingPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python File 읽어서 NetWork 객체 생성
- 모델 코드가 있는 위치를 읽을 수 있도록 path 추가
- 모델 import해서 객체를 생성 (model Class명이 Net인 것으로 가정)

In [2]:
import os
import sys
import importlib

def get_model(path):
  sys.path.append(os.path.dirname(path))

  net = importlib.import_module(os.path.basename(path).split('.')[0])
  model = net.Net()
  return model

# Weight 초기화

In [3]:
import torch

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
      super(CNN, self).__init__()
      self.conv1 = nn.Conv2d(
              in_channels=in_channels,
              out_channels=6,
              kernel_size=(3,3),
              stride=(1,1),
              padding=(1,1)
      )
      self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
      self.conv2 = nn.Conv2d(
              in_channels=6,
              out_channels=16,
              kernel_size=(3,3),
              stride=(1,1),
              padding=(1,1)
      )
      self.fc1 = nn.Linear(16*7*7, num_classes)
      # 예제의 핵심인 initialize_weights()로 __init__()이 호출될 때 실행됩니다.
      self.initialize_weights()

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.pool(x)
      x = F.relu(self.conv2(x))
      x = self.pool(x)
      x = x.reshape(x.shape[0], -1)
      x = self.fc1(x)
      
      return x

    def initialize_weights(self):
      for m in self.modules():
        if isinstance(m, nn.Conv2d):
          nn.init.kaiming_uniform_(m.weight)

          if m.bias is not None:
            nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.BatchNorm2d):
          nn.init.constant_(m.weight, 1)
          nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.Linear):
          nn.init.kaiming_uniform_(m.weight)
          nn.init.constant_(m.bias, 0)

if __name__ == '__main__':
    model = CNN(in_channels=3,num_classes=10)
    
    # He initialization과 Constant로 초기화 한것을 확인할 수 있습니다.
    for param in model.parameters():
        print(param)

Parameter containing:
tensor([[[[ 5.8590e-03,  9.4188e-02, -4.3836e-01],
          [ 2.6437e-02,  4.1553e-01, -4.9725e-02],
          [-4.1073e-01, -2.5424e-01,  4.6295e-01]],

         [[-8.1343e-02, -8.0433e-02,  2.9803e-01],
          [-3.9027e-01,  1.3963e-01, -4.5189e-01],
          [-8.1761e-02, -2.9941e-01, -8.6962e-02]],

         [[-2.3359e-01,  3.5392e-01,  7.9335e-02],
          [-4.0934e-01, -2.7547e-01, -4.7323e-02],
          [-4.3850e-01, -3.1184e-01, -2.3172e-01]]],


        [[[ 4.0410e-01, -4.2619e-01, -3.4129e-01],
          [ 1.2678e-01, -4.2520e-01,  9.3714e-02],
          [ 1.5808e-01, -4.6632e-02,  3.3886e-05]],

         [[ 3.3171e-01, -1.9735e-01,  1.5496e-01],
          [-2.3601e-01,  3.1578e-01,  1.9420e-01],
          [-4.2633e-01,  2.8305e-01, -8.4443e-02]],

         [[ 2.6075e-01,  2.8495e-01, -1.5672e-01],
          [-4.0373e-01, -4.5014e-01,  2.3347e-01],
          [ 2.5134e-01,  4.5762e-01, -2.1595e-01]]],


        [[[-1.5462e-01, -2.5606e-01, -3.3946

# load & save 방법
- state_dict는 dictionary다. 이 형태에 맞게 데이터를 쉽게 저장하거나 불러올 수 있다.
- state_dict에는 각 계층을 매개변수 Tensor로 매핑한다.
  - 매개변수 갖는 계층들 : conv layer, linear layer 등
- torch.optim 객체 또한 optimizer 상태, 사용된 하이퍼 파라미터 정보가 포함된 state_dict를 갖는다.
- inference를 위한 모델 저장 시 학습된 모델의 학습된 **매개변수만 저장**한다.
  - torch.save()
- model 저장 시 .pt or .pth 확장자를 사용하는 것이 일반적 규칙이며 tar를 통한 압축형태로 *.pth.tar 형태로 많이 사용한다.
- inference 용도로는 반드시 model.eval()을 통해 dropout, batchnormalization이 evaluation 모드로 설정되도록 해야한다.

In [7]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim # SGD, Adam etc..
import torch.nn.functional as F # param 필요없는 함수들
from torch.utils.data import DataLoader # dataset 꽌리 및 mini batch 생성 관련
import torchvision.datasets as datasets
import torchvision.transforms as transforms # dataset augmentation을 위한

In [10]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

def load_checkpoint(checkpoint, model, optimizer):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint["state_dict"])
  optimizer.load_state_dict(checkpoint["optimizer"])

def main():
  model = torchvision.models.vgg16(pretrained=False)
  optimizer = optim.Adam(model.parameters())

  checkpoint = {
      "state_dict" : model.state_dict(),
      "optimizer": optimizer.state_dict()
  }

  save_checkpoint(checkpoint)

  load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

if __name__ == "__main__":
  main()

=> Saving checkpoint
=> Loading checkpoint


# DataLoader 사용법

In [11]:
import torch
from torchvision import datasets, transforms

batch_size = 32
test_batch_size = 32

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root = "datasets/", # 현재 경로에 MNIST 데이터셋 생성 후 저장
        train = True,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean = (0.5,), std = (0.5,)) # normalize를 위한 mean과 std
        ])
    ),
    batch_size = batch_size,
    shuffle = True
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
# test 데이터를 사용하기 위한 test_loader 생성
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root = "datasets/", # 현재 경로에 datasets/MNIST/ 를 생성 후 데이터를 저장한다.
        train = False, # test 용도의 data 셋을 저장한다.
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(), # tensor 타입으로 데이터 변경
            transforms.Normalize(mean = (0.5,), std = (0.5,)) # train과 동일한 조건으로 normalize 한다.            
        ])
    ),
    batch_size = test_batch_size,
    shuffle = True
)

In [13]:
image, label = next(iter(train_loader)) # generator와 같이 동작하므로 next로 샘플을 생성할 수 있다.

In [14]:
image.shape, label.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

# Pre-Trained model 사용법
```python
checkpoint = {
    'state_dict' : model.state_dict(), 
    'optimizer': optimizer.state_dict(),
    'epoch' : epoch,
    'scheduler' : scheduler.state_dict(),
    'lr' : lr,
    'best_val', best_val
}
```
- 위와 같은 정보로 모델이 저장되어있다고 가정
- 위 6가지 정보는 모델의 학습을 계속 이어 나갈 때 꼭 필요한 정보다.
- 모델 저장 시 꼭 저장하는 것을 추천한다.

```python
resume_file_path = "../path/to/the/.../pre_trained.pth"
checkpoint = torch.load(resume_file_path)

model.load_state_dict(checkpoint['state_dict'])
start_epoch = checkpoint['epoch']
optimizer.load_state_dict(checkpoint['optimizer'])
scheduler.load_state_dict(checkpoint['scheduler'])
best_val = checkpoint['best_val']
```
- 모델 불러올 때, 위와 같이 사용한다.

# Pre-Trained model 수정 방법

```python
# pre-trained weight를 불러옵니다.
pretrained_weight= torch.load(weight_path)
```
- pretrained_weight에 collections.OrderedDict 타입으로 정보들이 저장된다.
- pretrained_weight의 key는 layer의 이름, value는 layer's weight 값이다.

```python
for i, key in enumerate(pretrained_weight.keys()):
    print("%d th, layer : %s" %(i, key))
```
- key값을 탐색해서 필요없는 layer를 찾는다.

- 1. **필요없는 layer 직접 제거 하는 방법**

```python
delete_layers = []
delete_layers.append("key value (layer name)")

for delete_layer in delete_layers:
    del pretrained_weight[delete_layer]
```

- **2. 필요없는 layer의 시작번호 입력하면 그 이후 모든 layer 제거하는 방법**

```python
# [0, delete_start_number) 범위의 layer만 남기고 나머지는 삭제합니다.
delete_start_number = 100
delete_layers = [key for i, key in enumerate(pretrained_weight.keys()) if i >= delete_start_number]
for delete_layer in delete_layers:
    del pretrained_weight[delete_layer]
```

# Checkpoint 값 변경 후 저장

```python
checkpoint = {
    'state_dict' : model.state_dict(), 
    'optimizer': optimizer.state_dict(),
    'epoch' : epoch,
    'scheduler' : scheduler.state_dict(),
    'lr' : lr,
    'best_val', best_val
}
```

```python
resume_file_path1 = "../path/to/the/.../pre_trained1.pth"
resume_file_path2 = "../path/to/the/.../pre_trained2.pth"
checkpoint1 = torch.load(resume_file_path1)
checkpoint2 = torch.load(resume_file_path2)

checkpoint1['state_dict'] = checkpoint2['state_dict']
torch.save(state, filename)
```

# Learning Rate Scheduler 사용법
```python
# model → optimizer → scheduler
model = Net().to(device)

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.1, paience = 5, verbose = True)

for epoch in range(10):
    train(...)
    val_loss = validate(...)
    # Note that step should be called after validate()
    scheduler.step(val_loss)
```
- model 선언 -> optmizre에 할당 -> 스케쥴러에 optimizer ㅎ할당
- 위 3개가 모두 엮이게 된다.
- 스케쥴러 업데이트는 보통 validation 과정 후 사용한다.
  - val loss를 입력으로 주면 그 loss 값과 스케쥴러 선언 시 사용한 옵션들을 통해 learning_rate를 dynamic하게 조절할 수 있다.

# model의 파라미터 확인 방법
```python
next(model.parameters())
# 바로 다음 값 1개 확인

for param in model.parameters():
    print(param)
```

# Tensor Deep Copy
- clone으로 복제 후 연결성을 끊어야한다.

In [16]:
A = torch.randn(10)
B = A.clone().detach()

# 일부 weight만 update 하는 방법
```python
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
model_dict = model.state_dict()

update_dict # update_dict is subset of model_dict

# 1. filter out unnecessary keys
filtered_update_dict = {k: v for k, v in update_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(filtered_update_dict) 
# 3. load the new state dict
model.load_state_dict(model_dict)
```
- 전체 weight를 갖고있는 dict와 update할 dict 두개를 준비
- model_dict.update(update_dict)를 통해 update할 weight를 덮어쓴다.